<a href="https://colab.research.google.com/github/RizanSM/zero_shot_llms_in_HIL_RL/blob/main/01_highway_env/01_generate_trajectories/01_generate_trajectories_highway.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install the required libraries in your Google Colab environment
!pip install stable-baselines3 gymnasium highway-env -q

In [ ]:
# Import the necessary libraries
import gymnasium as gym
import highway_env
import numpy as np
import pandas as pd
import pickle
import os
import matplotlib.pyplot as plt

In [ ]:
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.monitor import Monitor
from google.colab import data_table
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

In [ ]:
# Path to save the training logs
log_dir = "/content/drive/MyDrive/data_rp1/0_log_dir/0_ppo_highway_initial"                     # Update directory location 1
os.makedirs(log_dir, exist_ok=True)

In [ ]:
# THE ENVIRONMENT
# Step 1.1: Choose the Environment
# Initialize the environment.
env = gym.make('highway-v0')
env = Monitor(env, log_dir)

In [ ]:
# Step 1.2: Initial Observation
# Print out a sample observation to see what the agent receives at the start.
obs = env.reset()
print("Initial Observation: ", obs)

In [ ]:
# Step 1.3: Implement PPO with the Highway environment
# Initialize the PPO algorithm with the chosen environment.
# Create the PPO model with the Highway environment
model = PPO('MlpPolicy', env, verbose=1, tensorboard_log=log_dir)

In [ ]:
# Step 1.4: Train the model
# 10,000 timesteps for initial training
model.learn(total_timesteps=10000)

In [ ]:
log_path = os.path.join(log_dir, "monitor.csv")
df = pd.read_csv(log_path, skiprows=1)
df.rename(columns={"index": "episode", "r": "reward", "l": "length", "t": "time_step"}, inplace=True)

In [ ]:
# Step 1.5: Save the trained model to Google Drive
model.save('/content/drive/MyDrive/data_rp1/1_trained_models/0_ppo_highway_model_with_intial_training')                   # Update directory location 2

In [ ]:
# Step 1.6: Load the saved PPO model from Google Drive
model = PPO.load('/content/drive/MyDrive/data_rp1/1_trained_models/0_ppo_highway_model_with_intial_training')             # Update directory location 3

# TRAJECTORY GENERATION AND COLLECTION

In [ ]:
# Step-2: TRAJECTORY COLLECTION
# TRAJECTORY COLLECTION WITH ADDTIONNAL INFORMATION
# Initialize a list to store trajectory data
trajectories = []

# FUNCTION TO COLLECT TRAJECTORY DATA (state-action-reward transitions).

def collect_trajectory_data(env, model, num_episodes):
    """
    Collect trajectory data for a number of episodes.
    Each trajectory contains state-action-reward sequences.
    """
    trajectory_data = []

    for episode in range(num_episodes):
        state, _ = env.reset()  # Reset the environment at the start of each episode
        done = False
        episode_data = []

        while not done:
            # Get action from the trained PPO model
            action, _states = model.predict(state)

            # Take the action and get next state and reward
            next_state, reward, terminated, truncated, info = env.step(action)
            done = terminated or truncated
            # Extract lane index and collision flag
            lane_index = int(env.unwrapped.vehicle.lane_index[2])  # Assuming 'env.unwrapped.vehicle' gives access to the agent's vehicle object and 'lane_index' attribute within it contains the lane index.
            collision_flag = int(info.get('crashed', 0))  # Use get() with a default value to handle missing 'crashed' key.

            # Store the trajectory: (state, action, reward, next_state)
            episode_data.append({
                "state": state,
                "action": action,
                "reward": reward,
                "next_state": next_state,
                "lane_indices": lane_index,
                "collision_flags": collision_flag
            })

            # Update the state for the next iteration
            state = next_state

        # Add the episode data to the overall trajectory list
        trajectory_data.append(episode_data)

    return trajectory_data

In [ ]:
# Collect data for 100 episodes
trajectory_data = collect_trajectory_data(env, model, num_episodes=100)

In [ ]:
# FUNCTION TO PREPROCESS TRAJECTORY DATA
def preprocess_trajectory_data(trajectory_data):
    """
    Preprocesses the trajectory data into a structured format for further analysis.
    Returns a DataFrame with columns: episode, time_step, state, action, reward, next_state, speed, and reward_details.
    """
    processed_data = []

    for episode_num, episode_data in enumerate(trajectory_data):
        for time_step, step in enumerate(episode_data):
            # Flatten the state and next_state for easy interpretation (if they are multi-dimensional)
            state = np.array(step['state']).flatten()  # Flatten the state vector (if multi-dimensional)
            next_state = np.array(step['next_state']).flatten()  # Flatten the next_state vector

            collision_flag = step['collision_flags']
            lane_index = step['lane_indices']

            # Append the processed data for this step
            processed_data.append({
                "episode": episode_num,
                "time_step": time_step,
                "state": state,
                "action": step['action'],
                "reward": step['reward'],
                "next_state": next_state,
                "collision_flag": collision_flag,
                "lane_index": lane_index
            })

    # Convert the list of processed data into a DataFrame
    df = pd.DataFrame(processed_data)
    return df

In [ ]:
# Preprocess the trajectory data
trajectory_df = preprocess_trajectory_data(trajectory_data)

In [ ]:
data_table.enable_dataframe_formatter()
data_table.DataTable(trajectory_df)

In [ ]:
# Check the data type of each column
print(type(trajectory_df['episode'][0]))
print(type(trajectory_df['time_step'][0]))
print(type(trajectory_df['state'][0]))
print(type(trajectory_df['action'][0]))
print(type(trajectory_df['reward'][0]))
print(type(trajectory_df['next_state'][0]))
print(type(trajectory_df['collision_flag'][0]))
print(type(trajectory_df['lane_index'][0]))

In [ ]:
# Define the path to save the DataFrame (adjust the path as necessary)
trajectory_df_path = '/content/drive/MyDrive/data_rp1/2_trajectories/0_initial_training/initial_trajectory_df.csv'              # Update directory location 4
# Save the DataFrame to Google Drive as a CSV file
trajectory_df.to_csv(trajectory_df_path, index=False)

In [ ]:
# Save the processed dataframe as a pickle file
trajectory_df.to_pickle('/content/drive/MyDrive/data_rp1/2_trajectories/0_initial_training/0_initial_trajectory_df.pkl')        # Update directory location 5